# ShopEZ Delta Lake Assignment

In [0]:
# Basic imports & setup

from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import datetime, timedelta
import random

# Create and use a simple database (works even on free Databricks)
spark.sql("CREATE DATABASE IF NOT EXISTS shopez_db")
spark.sql("USE shopez_db")

table_name = "orders_delta"
small_files_table = "orders_small_files_demo"

print("Current database:", spark.sql("SELECT current_database()").collect()[0][0])
print("Main Delta table:", table_name)
print("Small-files demo table:", small_files_table)

# Clean up old tables if they exist (safe to re-run notebook)
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
spark.sql(f"DROP TABLE IF EXISTS {small_files_table}")

Current database: shopez_db
Main Delta table: orders_delta
Small-files demo table: orders_small_files_demo


DataFrame[]

## 1. Ingest sample order data into a Spark DataFrame

We will **generate synthetic data** to simulate daily e-commerce orders.

In [0]:
def generate_orders(num_days=7, orders_per_day=500):
    countries = ["US", "IN", "UK", "DE", "AU"]
    currencies = {"US": "USD", "IN": "INR", "UK": "GBP", "DE": "EUR", "AU": "AUD"}
    statuses = ["CREATED", "PAID", "CANCELLED"]

    data = []
    now = datetime.utcnow()

    order_counter = 1
    for day_offset in range(num_days):
        day = now - timedelta(days=day_offset)
        for _ in range(orders_per_day):
            country = random.choice(countries)
            currency = currencies[country]
            ts = day.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(
                seconds=random.randint(0, 24 * 60 * 60 - 1)
            )
            amount = round(random.uniform(5, 500), 2)
            status = random.choices(statuses, weights=[0.2, 0.7, 0.1], k=1)[0]

            data.append(
                (
                    f"O{order_counter:08d}",  # order_id
                    ts,
                    f"C{random.randint(1, 2000):06d}",  # customer_id
                    country,
                    float(amount),
                    currency,
                    status,
                )
            )
            order_counter += 1

    schema = T.StructType(
        [
            T.StructField("order_id", T.StringType(), False),
            T.StructField("order_timestamp", T.TimestampType(), False),
            T.StructField("customer_id", T.StringType(), False),
            T.StructField("country", T.StringType(), False),
            T.StructField("amount", T.DoubleType(), False),
            T.StructField("currency", T.StringType(), False),
            T.StructField("status", T.StringType(), False),
        ]
    )

    return spark.createDataFrame(data, schema)


orders_df = generate_orders(num_days=7, orders_per_day=500)  # ~3500 rows
orders_df.printSchema()
display(orders_df.limit(10))

root
 |-- order_id: string (nullable = false)
 |-- order_timestamp: timestamp (nullable = false)
 |-- customer_id: string (nullable = false)
 |-- country: string (nullable = false)
 |-- amount: double (nullable = false)
 |-- currency: string (nullable = false)
 |-- status: string (nullable = false)



order_id,order_timestamp,customer_id,country,amount,currency,status
O00000001,2025-12-01T19:19:37.000Z,C000724,UK,210.46,GBP,PAID
O00000002,2025-12-01T22:06:22.000Z,C001584,IN,206.34,INR,CANCELLED
O00000003,2025-12-01T16:15:45.000Z,C000301,AU,362.26,AUD,PAID
O00000004,2025-12-01T04:54:01.000Z,C001204,UK,249.84,GBP,CANCELLED
O00000005,2025-12-01T15:03:06.000Z,C001031,UK,477.57,GBP,PAID
O00000006,2025-12-01T20:56:31.000Z,C000065,AU,443.67,AUD,PAID
O00000007,2025-12-01T15:22:20.000Z,C001333,AU,331.04,AUD,CREATED
O00000008,2025-12-01T09:00:33.000Z,C000884,UK,21.86,GBP,CREATED
O00000009,2025-12-01T01:07:32.000Z,C000945,IN,473.18,INR,PAID
O00000010,2025-12-01T18:31:38.000Z,C001439,US,375.92,USD,PAID


## 2. Add derived column `order_date` (date from `order_timestamp`)

In [0]:
orders_with_date_df = orders_df.withColumn("order_date", F.to_date("order_timestamp"))

orders_with_date_df.printSchema()
display(orders_with_date_df.limit(10))

root
 |-- order_id: string (nullable = false)
 |-- order_timestamp: timestamp (nullable = false)
 |-- customer_id: string (nullable = false)
 |-- country: string (nullable = false)
 |-- amount: double (nullable = false)
 |-- currency: string (nullable = false)
 |-- status: string (nullable = false)
 |-- order_date: date (nullable = false)



order_id,order_timestamp,customer_id,country,amount,currency,status,order_date
O00000001,2025-12-01T19:19:37.000Z,C000724,UK,210.46,GBP,PAID,2025-12-01
O00000002,2025-12-01T22:06:22.000Z,C001584,IN,206.34,INR,CANCELLED,2025-12-01
O00000003,2025-12-01T16:15:45.000Z,C000301,AU,362.26,AUD,PAID,2025-12-01
O00000004,2025-12-01T04:54:01.000Z,C001204,UK,249.84,GBP,CANCELLED,2025-12-01
O00000005,2025-12-01T15:03:06.000Z,C001031,UK,477.57,GBP,PAID,2025-12-01
O00000006,2025-12-01T20:56:31.000Z,C000065,AU,443.67,AUD,PAID,2025-12-01
O00000007,2025-12-01T15:22:20.000Z,C001333,AU,331.04,AUD,CREATED,2025-12-01
O00000008,2025-12-01T09:00:33.000Z,C000884,UK,21.86,GBP,CREATED,2025-12-01
O00000009,2025-12-01T01:07:32.000Z,C000945,IN,473.18,INR,PAID,2025-12-01
O00000010,2025-12-01T18:31:38.000Z,C001439,US,375.92,USD,PAID,2025-12-01


## 3. Write the DataFrame as a **managed Delta table** partitioned by `country` and `order_date`

We use `.saveAsTable()` instead of a file path. Databricks chooses a secure storage location.

In [0]:
(
    orders_with_date_df.write.format("delta")
    .mode("overwrite")
    .partitionBy("country", "order_date")
    .saveAsTable(table_name)
)

print("Delta table created as managed table:", table_name)

Delta table created as managed table: orders_delta


## 4. Verify the partition structure (without DBFS access)

We use SQL commands to see partitioning and storage details:
* `DESCRIBE DETAIL`
* `SHOW PARTITIONS`

In [0]:
print("Table detail (includes location, format, partition columns):")
display(spark.sql(f"DESCRIBE DETAIL {table_name}"))

print("List of partitions:")
display(spark.sql(f"SHOW PARTITIONS {table_name}"))

Table detail (includes location, format, partition columns):


format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,9e132556-eadd-4975-b3e7-28d197b1c300,workspace.shopez_db.orders_delta,null,,2025-12-01T16:59:35.274Z,2025-12-01T16:59:42.000Z,"List(country, order_date)",List(),35,114765,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


List of partitions:


country,order_date
AU,2025-11-29
UK,2025-11-25
US,2025-12-01
AU,2025-11-28
DE,2025-11-29
UK,2025-11-30
IN,2025-11-26
AU,2025-11-30
IN,2025-11-28
UK,2025-11-28


## 5. Run queries that demonstrate **partition pruning**

We:
* Load the Delta table
* Filter on a specific `country` and `order_date`
* Use `.explain(True)` to see partition pruning in the plan

In [0]:
delta_df = spark.table(table_name)
display(delta_df.limit(5))

order_id,order_timestamp,customer_id,country,amount,currency,status,order_date
O00001502,2025-11-28T05:34:50.000Z,C000469,AU,218.44,AUD,PAID,2025-11-28
O00001506,2025-11-28T02:59:34.000Z,C000997,AU,313.38,AUD,PAID,2025-11-28
O00001507,2025-11-28T06:29:07.000Z,C000500,AU,97.63,AUD,CANCELLED,2025-11-28
O00001512,2025-11-28T07:06:04.000Z,C000501,AU,466.11,AUD,PAID,2025-11-28
O00001524,2025-11-28T05:17:32.000Z,C001086,AU,252.02,AUD,CANCELLED,2025-11-28


In [0]:
# Pick one `(country, order_date)` combination
sample_partition = (
    delta_df.select("country", "order_date").distinct().limit(1).collect()[0]
)
sample_country = sample_partition["country"]
sample_date = sample_partition["order_date"]

print("Sample partition:", sample_country, sample_date)

filtered_df = delta_df.filter(
    (F.col("country") == sample_country) & (F.col("order_date") == sample_date)
)

print("Filtered result (should hit only one partition):")
display(filtered_df)

print("Physical plan with partition pruning:")
filtered_df.explain(True)

Sample partition: AU 2025-11-28
Filtered result (should hit only one partition):


order_id,order_timestamp,customer_id,country,amount,currency,status,order_date
O00001502,2025-11-28T05:34:50.000Z,C000469,AU,218.44,AUD,PAID,2025-11-28
O00001506,2025-11-28T02:59:34.000Z,C000997,AU,313.38,AUD,PAID,2025-11-28
O00001507,2025-11-28T06:29:07.000Z,C000500,AU,97.63,AUD,CANCELLED,2025-11-28
O00001512,2025-11-28T07:06:04.000Z,C000501,AU,466.11,AUD,PAID,2025-11-28
O00001524,2025-11-28T05:17:32.000Z,C001086,AU,252.02,AUD,CANCELLED,2025-11-28
O00001530,2025-11-28T19:03:08.000Z,C001433,AU,30.65,AUD,CREATED,2025-11-28
O00001534,2025-11-28T00:08:09.000Z,C000485,AU,428.04,AUD,CREATED,2025-11-28
O00001535,2025-11-28T23:56:11.000Z,C000190,AU,140.77,AUD,PAID,2025-11-28
O00001540,2025-11-28T00:56:13.000Z,C000639,AU,297.46,AUD,PAID,2025-11-28
O00001546,2025-11-28T16:55:24.000Z,C001193,AU,351.45,AUD,PAID,2025-11-28


Physical plan with partition pruning:
== Parsed Logical Plan ==
'Filter 'and('`==`('country, AU), '`==`('order_date, 2025-11-28))
+- 'UnresolvedRelation [orders_delta], [], false

== Analyzed Logical Plan ==
order_id: string, order_timestamp: timestamp, customer_id: string, country: string, amount: double, currency: string, status: string, order_date: date
Filter ((country#11805 = AU) AND (order_date#11809 = 2025-11-28))
+- SubqueryAlias workspace.shopez_db.orders_delta
   +- Relation workspace.shopez_db.orders_delta[order_id#11802,order_timestamp#11803,customer_id#11804,country#11805,amount#11806,currency#11807,status#11808,order_date#11809] parquet

== Optimized Logical Plan ==
Filter (((isnotnull(order_date#11809) AND (order_date#11809 = 2025-11-28)) AND isnotnull(country#11805)) AND (country#11805 = AU))
+- Relation workspace.shopez_db.orders_delta[order_id#11802,order_timestamp#11803,customer_id#11804,country#11805,amount#11806,currency#11807,status#11808,order_date#11809] parquet

## 6. Demonstrate **Delta Lake Time Travel**

Because this is a managed Delta table, we use:
* `DESCRIBE HISTORY`
* `VERSION AS OF` in SQL
* Or `option("versionAsOf", ...)` in Spark

In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, table_name)

print("History BEFORE update:")
display(delta_table.history())

History BEFORE update:


version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2025-12-01T16:59:42.000Z,73385669917309,agnipradeep19@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""country"",""order_date""], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(3636204605898368),1201-164734-6v5jb4x0-v2n,null,WriteSerializable,false,"Map(numFiles -> 35, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 3500, numOutputBytes -> 114765)",null,Databricks-Runtime/17.2.x-aarch64-photon-scala2.13


In [0]:
# Update: set all PAID orders in one country to CANCELLED (for demo)

update_country = sample_country

delta_table.update(
    condition=(F.col("country") == update_country) & (F.col("status") == "PAID"),
    set={"status": F.lit("CANCELLED")},
)

print("History AFTER update:")
display(delta_table.history())

print("Current data for updated country:")
display(
    delta_df.filter(F.col("country") == update_country)
    .select("order_id", "country", "status")
    .limit(20)
)

History AFTER update:


version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-12-01T17:00:31.000Z,73385669917309,agnipradeep19@gmail.com,UPDATE,"Map(predicate -> [""((country#11967 = AU) AND (status#11970 = PAID))""])",null,List(3636204605898368),1201-164734-6v5jb4x0-v2n,0,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 7, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3493, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1495, numAddedFiles -> 7, numUpdatedRows -> 496, numAddedBytes -> 20943, rewriteTimeMs -> 1956)",null,Databricks-Runtime/17.2.x-aarch64-photon-scala2.13
0,2025-12-01T16:59:42.000Z,73385669917309,agnipradeep19@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""country"",""order_date""], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(3636204605898368),1201-164734-6v5jb4x0-v2n,null,WriteSerializable,false,"Map(numFiles -> 35, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 3500, numOutputBytes -> 114765)",null,Databricks-Runtime/17.2.x-aarch64-photon-scala2.13


Current data for updated country:


order_id,country,status
O00001507,AU,CANCELLED
O00001524,AU,CANCELLED
O00001530,AU,CREATED
O00001534,AU,CREATED
O00001548,AU,CREATED
O00001550,AU,CREATED
O00001608,AU,CANCELLED
O00001617,AU,CREATED
O00001618,AU,CANCELLED
O00001620,AU,CANCELLED


In [0]:
# Read an older version (before the update) using time travel
# Version 0 is the initial write; version 1 is after the update.

version_0_df = (
    spark.read.format("delta")
    .option("versionAsOf", 0)
    .table(table_name)  # NOTE: table() + option works on managed Delta
)

print("Data from version 0 (before update) for same country:")
display(
    version_0_df.filter(F.col("country") == update_country)
    .select("order_id", "country", "status")
    .limit(20)
)

Data from version 0 (before update) for same country:


order_id,country,status
O00001502,AU,PAID
O00001506,AU,PAID
O00001507,AU,CANCELLED
O00001512,AU,PAID
O00001524,AU,CANCELLED
O00001530,AU,CREATED
O00001534,AU,CREATED
O00001535,AU,PAID
O00001540,AU,PAID
O00001546,AU,PAID


## 7. Demonstrate **Schema Evolution**

New columns:
* `payment_method` (e.g., CARD, UPI, COD)
* `coupon_code` (nullable string)

We create a new batch of data with the extra columns and append with `mergeSchema = true`.

In [0]:
def generate_orders_with_new_schema(num_days=3, orders_per_day=300):
    countries = ["US", "IN", "UK", "DE", "AU"]
    currencies = {"US": "USD", "IN": "INR", "UK": "GBP", "DE": "EUR", "AU": "AUD"}
    statuses = ["CREATED", "PAID", "CANCELLED"]
    payment_methods = ["CARD", "UPI", "COD", "WALLET", "NETBANKING"]
    coupons = [None, "NEW10", "SALE20", "FREESHIP", "LOYALTY5"]

    data = []
    now = datetime.utcnow()

    order_counter = 100000  # separate range to avoid clashes
    for day_offset in range(num_days):
        day = now + timedelta(days=day_offset + 1)
        for _ in range(orders_per_day):
            country = random.choice(countries)
            currency = currencies[country]
            ts = day.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(
                seconds=random.randint(0, 24 * 60 * 60 - 1)
            )
            amount = round(random.uniform(5, 500), 2)
            status = random.choices(statuses, weights=[0.2, 0.7, 0.1], k=1)[0]
            payment_method = random.choice(payment_methods)
            coupon_code = random.choice(coupons)

            data.append(
                (
                    f"O{order_counter:08d}",
                    ts,
                    f"C{random.randint(1, 2000):06d}",
                    country,
                    float(amount),
                    currency,
                    status,
                    payment_method,
                    coupon_code,
                )
            )
            order_counter += 1

    schema_new = T.StructType(
        [
            T.StructField("order_id", T.StringType(), False),
            T.StructField("order_timestamp", T.TimestampType(), False),
            T.StructField("customer_id", T.StringType(), False),
            T.StructField("country", T.StringType(), False),
            T.StructField("amount", T.DoubleType(), False),
            T.StructField("currency", T.StringType(), False),
            T.StructField("status", T.StringType(), False),
            T.StructField("payment_method", T.StringType(), True),
            T.StructField("coupon_code", T.StringType(), True),
        ]
    )

    return spark.createDataFrame(data, schema_new)


new_orders_df = generate_orders_with_new_schema(num_days=3, orders_per_day=300)
new_orders_df = new_orders_df.withColumn("order_date", F.to_date("order_timestamp"))

new_orders_df.printSchema()
display(new_orders_df.limit(10))

root
 |-- order_id: string (nullable = false)
 |-- order_timestamp: timestamp (nullable = false)
 |-- customer_id: string (nullable = false)
 |-- country: string (nullable = false)
 |-- amount: double (nullable = false)
 |-- currency: string (nullable = false)
 |-- status: string (nullable = false)
 |-- payment_method: string (nullable = true)
 |-- coupon_code: string (nullable = true)
 |-- order_date: date (nullable = false)



order_id,order_timestamp,customer_id,country,amount,currency,status,payment_method,coupon_code,order_date
O00100000,2025-12-02T22:54:10.000Z,C001946,AU,10.8,AUD,PAID,COD,null,2025-12-02
O00100001,2025-12-02T00:17:30.000Z,C001960,DE,187.14,EUR,CREATED,WALLET,SALE20,2025-12-02
O00100002,2025-12-02T21:55:53.000Z,C000495,US,382.66,USD,PAID,COD,LOYALTY5,2025-12-02
O00100003,2025-12-02T02:07:58.000Z,C000474,IN,350.64,INR,PAID,UPI,SALE20,2025-12-02
O00100004,2025-12-02T18:36:39.000Z,C001807,DE,466.17,EUR,CANCELLED,COD,LOYALTY5,2025-12-02
O00100005,2025-12-02T18:46:50.000Z,C000495,US,413.45,USD,PAID,UPI,SALE20,2025-12-02
O00100006,2025-12-02T13:21:25.000Z,C000442,US,183.62,USD,PAID,CARD,NEW10,2025-12-02
O00100007,2025-12-02T10:39:54.000Z,C001501,DE,379.52,EUR,PAID,NETBANKING,null,2025-12-02
O00100008,2025-12-02T08:25:46.000Z,C001055,AU,114.56,AUD,PAID,WALLET,NEW10,2025-12-02
O00100009,2025-12-02T02:18:48.000Z,C000589,DE,430.66,EUR,PAID,NETBANKING,FREESHIP,2025-12-02


In [0]:
(
    new_orders_df.write.format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .partitionBy("country", "order_date")
    .saveAsTable(table_name)
)

print("Appended new data with schema evolution into table:", table_name)

delta_df = spark.table(table_name)
delta_df.printSchema()
display(delta_df.limit(20))

Appended new data with schema evolution into table: orders_delta
root
 |-- order_id: string (nullable = true)
 |-- order_timestamp: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- status: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- coupon_code: string (nullable = true)



order_id,order_timestamp,customer_id,country,amount,currency,status,order_date,payment_method,coupon_code
O00002508,2025-11-26T14:34:40.000Z,C000142,IN,308.89,INR,CREATED,2025-11-26,null,null
O00002510,2025-11-26T05:43:37.000Z,C001150,IN,242.03,INR,PAID,2025-11-26,null,null
O00002511,2025-11-26T07:21:05.000Z,C001713,IN,190.43,INR,PAID,2025-11-26,null,null
O00002517,2025-11-26T19:59:04.000Z,C000147,IN,345.38,INR,PAID,2025-11-26,null,null
O00002520,2025-11-26T00:32:41.000Z,C001716,IN,38.56,INR,PAID,2025-11-26,null,null
O00002528,2025-11-26T15:20:27.000Z,C001439,IN,338.92,INR,CREATED,2025-11-26,null,null
O00002542,2025-11-26T00:49:23.000Z,C001628,IN,45.46,INR,PAID,2025-11-26,null,null
O00002552,2025-11-26T21:29:17.000Z,C000048,IN,231.38,INR,CREATED,2025-11-26,null,null
O00002563,2025-11-26T05:22:14.000Z,C001014,IN,58.53,INR,PAID,2025-11-26,null,null
O00002568,2025-11-26T06:56:31.000Z,C000531,IN,321.56,INR,CREATED,2025-11-26,null,null


## 8. Demonstrate **Updates & Deletes** using DeltaTable

* UPDATE: mark some PAID orders as CANCELLED for a country.
* DELETE: remove orders below a certain amount.

In [0]:
delta_table = DeltaTable.forName(spark, table_name)

target_country = "IN"

# UPDATE
delta_table.update(
    condition=(F.col("country") == target_country)
    & (F.col("status") == "PAID")
    & (F.col("amount") < 50),
    set={"status": F.lit("CANCELLED")},
)

print("Updated low-value PAID orders to CANCELLED for country:", target_country)

display(
    spark.table(table_name)
    .filter(F.col("country") == target_country)
    .select("order_id", "country", "amount", "status")
    .orderBy(F.col("amount").asc())
    .limit(20)
)

Updated low-value PAID orders to CANCELLED for country: IN


order_id,country,amount,status
O00100139,IN,6.12,CANCELLED
O00000507,IN,6.57,CANCELLED
O00002231,IN,7.58,CANCELLED
O00001408,IN,9.49,CANCELLED
O00002578,IN,9.56,CANCELLED
O00002301,IN,10.05,CANCELLED
O00003109,IN,10.3,CREATED
O00001184,IN,10.45,CANCELLED
O00001990,IN,11.14,CANCELLED
O00000621,IN,11.3,CANCELLED


In [0]:
# DELETE
cutoff_amount = 10.0
delta_table.delete(F.col("amount") < cutoff_amount)

print(f"Deleted orders with amount < {cutoff_amount} (test data cleanup).")

delta_df = spark.table(table_name)
display(delta_df.orderBy(F.col("amount").asc()).limit(20))

Deleted orders with amount < 10.0 (test data cleanup).


order_id,order_timestamp,customer_id,country,amount,currency,status,order_date,payment_method,coupon_code
O00002301,2025-11-27T16:46:05.000Z,C001432,IN,10.05,INR,CANCELLED,2025-11-27,null,null
O00000435,2025-12-01T13:48:43.000Z,C001955,UK,10.18,GBP,PAID,2025-12-01,null,null
O00001521,2025-11-28T08:11:17.000Z,C001447,US,10.26,USD,PAID,2025-11-28,null,null
O00003031,2025-11-25T12:37:16.000Z,C001613,AU,10.28,AUD,CREATED,2025-11-25,null,null
O00003109,2025-11-25T01:05:43.000Z,C000229,IN,10.3,INR,CREATED,2025-11-25,null,null
O00002659,2025-11-26T12:24:45.000Z,C001864,UK,10.41,GBP,PAID,2025-11-26,null,null
O00001184,2025-11-29T06:33:15.000Z,C000178,IN,10.45,INR,CANCELLED,2025-11-29,null,null
O00002456,2025-11-27T02:38:04.000Z,C000937,UK,10.63,GBP,PAID,2025-11-27,null,null
O00000956,2025-11-30T23:47:22.000Z,C001940,US,10.7,USD,CREATED,2025-11-30,null,null
O00100000,2025-12-02T22:54:10.000Z,C001946,AU,10.8,AUD,PAID,2025-12-02,COD,null


## 9. (Bonus) Optimize the table (if supported)

On some free workspaces, `OPTIMIZE` may be restricted.
We try it, but catch errors so the notebook still runs.

In [0]:
try:
    print("Running OPTIMIZE with ZORDER on customer_id ...")
    spark.sql(f"""
    OPTIMIZE {table_name}
    ZORDER BY (customer_id)
    """)
    print("OPTIMIZE completed.")
except Exception as e:
    print("OPTIMIZE failed or not supported on this workspace:")
    print(str(e))

Running OPTIMIZE with ZORDER on customer_id ...
OPTIMIZE completed.
